In [1]:
%load_ext dotenv
%dotenv .env 

from langchain_openai import ChatOpenAI

In [2]:
reasoning = {
    "effort": "low",
    "summary": None
}
test_model = ChatOpenAI(model="gpt-5-mini", reasoning=reasoning)

In [22]:
prompt = (
    "Two agents are tasked with manipulating blocks arranged in stacks on a table, from one configuration to another. "
    "Each block is uniquely labelled by a letter. Both agents can only interact with blocks at the top of each stack, "
    "and only interact with one block at a time. Additionally, one agent can only interact with vowel blocks, "
    "and the other can only interact with consonant blocks. Initially there are blocks A, B, C, D, E, F, G, I. D, E are directly on the table. "
    "C is on top of D, B is on top of C, A is on top of B, G is on top of E, I is on top of G, and F is on top of I. At any point, each agent can only be holding at most one block, and the other agent" 
    "cannot put a block on top of it if it's being held. "
    "The goal is to have F and A directly on the table, with C on top of F, I on top of C, G on top of I, E is on top of D, D on top of B, and B on top of A."
    "Finally, there cannot be more than 3 stacks on the table at any time, and the height of each stack can never exceed 4, and both agents know this."
    "Come up with a step-by-step plan to achieve this goal, specifying which agent does which action. Just do it, don't ask for confirmation."
    )

In [4]:
prompt = """The LPG planner is run with the following domain and problem definitions.
Domain:
(define (domain travel)
  (:requirements :strips :typing)
  (:types location number)
  (:predicates 
    (at ?loc - location)
    (has-flight ?from ?to - location)
    (visited ?loc - location)
    (at-day ?day - number)
    (next-day-sequence ?current ?next - number)
    (days-in-location ?loc - location ?days - number))
  (:action fly
    :parameters (?from ?to - location)
    :precondition (and (at ?from) (has-flight ?from ?to))
    :effect (and (not (at ?from)) (at ?to) (visited ?to)))
  (:action next-day
    :parameters (?current - number ?next - number)
    :precondition (and (at-day ?current) (next-day-sequence ?current ?next))
    :effect (and (not (at-day ?current)) (at-day ?next)))
  (:action stay
    :parameters (?loc - location ?days - number)
    :precondition (and (at ?loc) (at-day 1)) 
    :effect (days-in-location ?loc ?days)))

Problem:
(define (problem travel-planning)
  (:domain travel)
  (:objects
    london bucharest reykjavik - location
    1 2 3 4 5 6 7 8 9 10 11 12 13 14 - number)
  (:init
    (at london)
    (at-day 1)
    (next-day-sequence 1 2)
    (next-day-sequence 2 3)
    (next-day-sequence 3 4)
    (next-day-sequence 4 5)
    (next-day-sequence 5 6)
    (next-day-sequence 6 7)
    (next-day-sequence 7 8)
    (next-day-sequence 8 9)
    (next-day-sequence 9 10)
    (next-day-sequence 10 11)
    (next-day-sequence 11 12)
    (next-day-sequence 12 13)
    (next-day-sequence 13 14)
    (has-flight london bucharest)
    (has-flight reykjavik london)
    (has-flight london reykjavik))
  (:goal
    (and 
      (visited london)
      (visited bucharest)
      (visited reykjavik)
      (at-day 14)
      (days-in-location london 4)
      (days-in-location bucharest 5)
      (days-in-location reykjavik 7))))

It failed to produce a plan. Here is the log message:
Parsing error: Expected ':init', found ':'  (at char 56), (line:3, col:4)

Describe the error and revise the pddl domain and problem definitions to fix it.
Enclose the revised definitions with <domain>...</domain> and <problem>...</problem> tags respectively.
    
    """

In [5]:
response = test_model.invoke(prompt)

In [6]:
print(response.text())

Error summary
- The parser error arose because the problem file used bare numeric tokens (1 2 3 ...) as object names. In PDDL object names must be valid symbols (typically starting with a letter); bare numeric tokens are not valid identifiers and break the parser. That caused the parser to get out of sync and report "Expected ':init', found ':'".
- Also a few predicate/parameter names were ambiguous for counts/days. I revised the domain so days (calendar positions) and day-counts are explicit typed objects, and fixed the stay action parameters so it uses a day object in its precondition and a count object in its effect.

Revised domain and problem
<domain>
(define (domain travel)
  (:requirements :strips :typing)
  (:types location day count)
  (:predicates 
    (at ?loc - location)
    (has-flight ?from ?to - location)
    (visited ?loc - location)
    (at-day ?d - day)
    (next-day-sequence ?current ?next - day)
    (days-in-location ?loc - location ?c - count))
  (:action fly
    :

In [ ]:
import unified_planning as up
from unified_planning.io import PDDLReader
from unified_planning.shortcuts import OneshotPlanner
up.shortcuts.get_environment().credits_stream = None
from pathlib import Path

ACTOR_OUTPUT_PATH = "./tmp"
folder_name = "TripPlanner"

base_dir = (
        Path().parent / (ACTOR_OUTPUT_PATH + f"/{folder_name}")
    ).resolve()

dom, prob = base_dir / "refined_domain.pddl", base_dir / "refined_problem.pddl"   # put your paths here
try:
    problem = PDDLReader().parse_problem(dom, prob)
except Exception as e:
    print(f"Parsing error: {e}")
with OneshotPlanner(name="lpg") as planner:
    result = planner.solve(problem)
    print("Status:", result.status)
    if result.plan: 
        print(result.plan)